In [1]:
import numpy as np
import netket as nk
import functions

/home/keisuke/Documents/Research/sshfs/DimerMaster/functions.py:65: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec)


In [3]:
from conf import *

In [2]:
from numba import njit

@njit
def vec_n_to_state(number, local_states, out):
    for n in range(number.shape[0]):
        out[n] = nk.operator._local_operator._number_to_state(number[n] , local_states, out[n])

In [3]:
N = 2**16
length = [2, 4]
L = 2 * np.prod(length)
hexagonal = nk.machine.graph_hex(length)
number = np.arange(N)
local_state = np.array([-1,1],dtype=np.int64)
out = np.empty((N,L), dtype=np.int64)
vec_n_to_state(number, local_state, out)
dimer_basis = out[hexagonal.is_dimer_basis2(out)]

In [4]:
e, ec, ce, cec = hexagonal.for_transition()

In [5]:
e[ec==-1]

array([[ 4,  8],
       [ 6, 10],
       [12,  0],
       [14,  2]])

In [6]:
P_0 = np.ones(dimer_basis.shape[0])
P_0 = P_0/np.sum(P_0)
v = P_0/np.sqrt(np.sum(P_0 ** 2))

In [7]:
s = np.random.choice(len(P_0), 100, replace=True, p=P_0)
dimer_basis[s]

array([[ 1,  1, -1, ...,  1,  1,  1],
       [ 1,  1, -1, ...,  1,  1,  1],
       [-1,  1,  1, ...,  1, -1, -1],
       ...,
       [-1, -1, -1, ..., -1,  1,  1],
       [-1,  1, -1, ...,  1,  1, -1],
       [ 1, -1, -1, ..., -1,  1,  1]])

In [8]:
h = 1
q = 1
V = h * q
# V = 1
sigmaz = np.array([[1, 0], [0, -1]])
sigmax = np.array([[0,1],[1,0]])
sigmay = np.array([[0,1],[-1,0]])

mszsz = np.kron(sigmaz, sigmaz)
mszsx = np.kron(sigmax, sigmax)
mszsy = np.kron(sigmay, sigmay)

potential_anti = (np.identity(4) + mszsz)/2
potential_ferro = (np.identity(4) - mszsz)/2

hexagon = nk.machine.graph_hex(length = length)


g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)
op = nk.operator.LocalOperator(hi)


In [9]:
e, ec, ce, cec = hexagon.for_transition()

In [10]:
hi = nk.hilbert.Spin(s=0.5, graph=g)
op = nk.operator.DimerLocalOperator(hi)

for edge, edge_color, pe, pec in zip(e, ec, ce, cec):
    l_op = -h*mszsx
    
    l_op = np.kron(np.identity(2), l_op)
    l_op = np.kron(l_op, np.identity(2))
    
    mat = []
    edge_ = []
    for p, c in zip(pe, pec):
        mat.append(np.kron(
            potential_ferro if c[0] == 1 else potential_anti,
            potential_ferro if c[1] == 1 else potential_anti,
        ))
        edge_.append(p[0].tolist() + p[1].tolist())
    
    op += nk.operator.LocalOperator(hi, l_op @ mat[0] + V * mat[0], edge_[0])
    op += nk.operator.LocalOperator(hi, l_op @ mat[1] + V * mat[1], edge_[1])

In [11]:
alpha = 4
ma = nk.machine.RbmDimer(hi, alpha = alpha, symmetry = False
                        ,use_hidden_bias = False, use_visible_bias = False, dtype=float)
ma.init_random_parameters(seed=1234, sigma=0.01)

In [12]:
sa = nk.sampler.DimerMetropolisLocal(machine=ma, n_chains=10, sweep_size= 20,length=length)
sr = nk.optimizer.SR(ma, diag_shift=0.05)
opt = nk.optimizer.Sgd(ma, learning_rate=0.01)
gs = nk.Vmc(
hamiltonian=op,
sampler=sa,
optimizer=opt,
n_samples=1000,
sr = sr,
n_discard=200
)
gs.run(n_iter=300)

/home/keisuke/Documents/Research/sshfs/DimerMaster/netket/machine/rbm.py:173: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 2d, A))
  r = x.dot(W)
  0%|          | 0/300 [00:00<?, ?it/s]

No output specified (out=[apath|nk.logging.JsonLogger(...)]).Running the optimization but not saving the output.


 11%|█         | 33/300 [00:07<01:01,  4.35it/s, Energy=0.00034+0.00000j ± 0.00041 [σ²=0.00014, R̂=0.9982]] 


KeyboardInterrupt: 

In [81]:
ma.log_val(dimer_basis)

array([0.04741705+0.j, 0.04796538+0.j, 0.04745201+0.j, 0.04737368+0.j,
       0.04695419+0.j, 0.04680067+0.j, 0.046157  +0.j, 0.04659263+0.j,
       0.04694302+0.j, 0.04618711+0.j, 0.04685424+0.j, 0.04656629+0.j,
       0.04616666+0.j, 0.04508593+0.j, 0.04598874+0.j, 0.04668497+0.j,
       0.04602444+0.j, 0.04547165+0.j, 0.04701593+0.j, 0.04543277+0.j,
       0.04628851+0.j, 0.04521021+0.j, 0.04622782+0.j, 0.04548385+0.j,
       0.04620717+0.j, 0.04680037+0.j, 0.04724961+0.j, 0.04743695+0.j,
       0.04743695+0.j, 0.04724961+0.j, 0.04680037+0.j, 0.04620717+0.j,
       0.04548385+0.j, 0.04622782+0.j, 0.04521021+0.j, 0.04628851+0.j,
       0.04543277+0.j, 0.04701593+0.j, 0.04547165+0.j, 0.04602444+0.j,
       0.04668497+0.j, 0.04598874+0.j, 0.04508593+0.j, 0.04616666+0.j,
       0.04656629+0.j, 0.04685424+0.j, 0.04618711+0.j, 0.04694302+0.j,
       0.04659263+0.j, 0.046157  +0.j, 0.04680067+0.j, 0.04695419+0.j,
       0.04737368+0.j, 0.04745201+0.j, 0.04796538+0.j, 0.04741705+0.j])

In [77]:
from numba import jitclass, int64, float64, complex128, njit, prange





get_conn = nk.operator.LocalOperator._get_conn_flattened_kernel


class dynamics: 
    def __init__(self,
                local_states,
                basis,
                constant,
                diag_mels,
                n_conns,
                mels,
                x_prime,
                acting_on,
                acting_size,):
        
        self.local_states = np.sort(local_states)
        self.basis = basis
        self.constant = constant
        self.diag_mels = diag_mels
        self.n_conns = n_conns
        self.mels = mels
        self.x_prime = x_prime
        self.acting_on = acting_on
        self.acting_size = acting_size
        
    
    
    def dynamics(self, X, time_list, E0):
        
        
        
        
        return self._dynamics(
            X, 
            time_list, 
            E0,
            self.local_states,
            self.basis,
            self.constant,
            self.diag_mels,
            self.n_conns,
            self.mels,
            self.x_prime,
            self.acting_on,
            self.acting_size,        
        )
    
    
    @staticmethod
    @njit
    def _dynamics(
            X,
            time_list,
            E0,
            _local_states,
            _basis,
            _constant,
            _diag_mels,
            _n_conns,
            _mels,
            _x_prime,
            _acting_on,
            _acting_size):
        
        # basis is float64[:]
        
        
         
        t_d = time_list[1]-time_list[0]
        t_end = np.shape(time_list)[0]
        p_array = np.zeros((X.shape[0],t_end,X.shape[1]),dtype= np.float64)
        p = np.zeros_like(p_array[0])
        t_s = time_list[0]
        
        for j in range(X.shape[0]):
            p = np.zeros_like(p_array[0])
            x = X[j]
            time = 0
            t_index_b = -1
        
            while True:
                x_prime, mels = get_conn(
                                    x.reshape((1, -1)),
                                    np.ones(1),
                                    _local_states,
                                    _basis,
                                    _constant,
                                    _diag_mels,
                                    _n_conns,
                                    _mels,
                                    _x_prime,
                                    _acting_on,
                                    _acting_size)

                mels = np.real(mels)
                n_conn = mels.shape[0]

                a_0 = mels[0] - E0
                r_1 = np.random.uniform(0,1)
                r_2 = np.random.uniform(0,1)
                tau = np.log(1/r_1)/a_0
                time += tau
                if time > t_s:
                    t_index = int((time-t_s) // t_d)
                    if t_index >= t_end - 1:
                        p[np.arange(t_index_b + 1, t_end)] = x
    #                     for i in range(t_index_b + 1, t_end):
    #                         p[i,:] = x
    #                     return p
                        break
    #                 for i in range(t_index_b + 1, t_index+1):　
    #                     p[i,:] = x
                    p[np.arange(t_index_b + 1, t_index+1)] = x

                    t_index_b = t_index

                s = 0

                for i in range(n_conn-1):
                    s -= mels[i + 1]
                    if s >= r_2 * a_0:
                        x = x_prime[i]
                        break
    #                     print(x_prime[i])
    #                     print(x)
            p_array[j] = p
                
        return p_array
            
        
    def run(self, basis, t_list, E_0, qout):
        
        out = self.dynamics(basis, t_list, E_0)
        
        qout.put(out)
            
            
        

In [72]:
from functions import dynamics

ImportError: cannot import name 'dynamics' from 'functions' (/home/keisuke/Documents/Research/sshfs/DimerMaster/functions.py)

In [78]:
d = dynamics(
            op._local_states,
            op._basis,
            op._constant,
            op._diag_mels,
            op._n_conns,  
            op._mels,
            op._x_prime,
            op._acting_on,
            op._acting_size,
            )


In [79]:
t_list = np.linspace(0, 10, 101)

In [86]:
basis = np.resize(np.ones(16), (100000, 16)).astype('float64')

In [ ]:
import multiprocessing as mp
b = mp.Queue()
p = mp.Process(target=d.run, args=(basis, t_list, 0,q))
p.start()
a = q.get()
print('got output')


In [ ]:
a = q.get()

In [76]:
a = d.dynamics(basis, t_list, 0)

In [20]:
import multiprocessing as mp

In [22]:
p = mp.Process(target=d.dynamics, args=(basis, t_list, 0,))

In [23]:
p.start()

In [ ]:
def dimer_corr (rIJ, aIJ, hexagon):
    

In [257]:
class new_hex:
    
    def __init__(self, l = np.array([4, 2])):
        
        self.a1 = self.a(np.float32(0))
        self.a2 =  self.a(np.pi*(5/3))
        self.R1 = self.a1 * l[0]
        self.R2 = self.a2 * l[1]
        
        self.epsilon = 1e-5
        
        self.l = l
        
        self.x_array = np.zeros((l[0],l[1],2)).astype(np.float32)
        
        for i in range(l[0]):
            for j in range(l[1]):
                self.x_array[i, j] = self.a1 * i + self.a2 * j
        
        self.x = np.zeros((np.prod(l),2), dtype=np.float32)
        
        for i in range(l[0]):
            for j in range(l[1]):
                self.x[j * l[0] + i] = self.x_array[i, j]
        
        self.lattice_pos_array = np.zeros([l[0], l[1], 6, 2],dtype=np.float32)
        
        
        
        
        for j in range(l[1]):
            for i in range(l[0]):
                for a in range(6):
                    self.lattice_pos_array[i, j, a] = self.x_array[i, j] + self.alpha(a)
                    
        self.lattice_pos_array = self.ProcessPeriodic(self.lattice_pos_array)
                    

                    
        self.lattice_pos = np.zeros((np.prod(l) * 2, 2), dtype=np.float32)
        
        for j in range(l[1]):
            for n,a in enumerate([0,5]):
                for i in range(l[0]):
                    self.lattice_pos[j * 2 * l[0] + l[0] * n + i] = self.lattice_pos_array[i, j, a] 
                    

        self.all_hex_index = []

        for i in range(l[0]):
            for j in range(l[1]):
                self.all_hex_index.append([i,j])
        self.all_hex_index = np.array(self.all_hex_index)

        self.edges , self.edges_color = self.edges_from_hex(l = self.all_hex_index, color=True, num =True)
        self.edges = np.sort(self.edges[:,np.array([0,5,4]),:].reshape(-1,2),axis=1)
        self.edges_color = self.edges_color[:,np.array([0,5,4])].reshape(-1)
        
                    
        
                
                
    def a(self, theta):
        return self.Rotation(theta) @ np.array([1,0])
    
    def alpha(self, i):
        r = np.array([0, (1/(np.sqrt(3)))],dtype=np.float32)
        
        return self.Rotation(np.pi*(1/3) * i) @ r
    
    def LatticeToHexIndex(self, X):
        
        lattice_pos_array = self.ProcessPeriodic(self.lattice_pos_array)
        X_ = self.ProcessPeriodic(X).reshape(-1, 2)
        
        HexIndex = np.zeros((X_.shape[0],3,2), dtype=np.int)
        
        for n, x in enumerate(X_):
            m = 0
            for j in range(self.l[1]):
                for i in range(self.l[0]):
                    lp = lattice_pos_array[i, j]
                    if (np.abs(lp-x).sum(axis=1) < self.epsilon).any():
                        HexIndex[n,m] = np.array([i, j])
                        m += 1
        
        
        return HexIndex.reshape(X.shape[:-1] + (3,) + (2,))
    
    def ProcessPeriodic(self, X):
        
        X_ = X.reshape(-1, 2).copy()
        
        W = self.to_lattice_vec(X_) % self.l
        
        A = np.concatenate((self.a1.reshape(-1,1), self.a2.reshape(-1,1)), axis=1)
        
#         for n in range(X_.shape[0]):

#             w = self.decompose(X_[n].copy(), self.R1, self.R2)

#             X_[n] = ((w[0] % 1)*self.R1 + (w[1] % 1)*self.R2)
            
        return (A @ W.T).T.reshape(X.shape)
    
    
    def edges_from_hex(self, l, color=False, num = True):
        
        '''
        
        l : coordinate number of hex
        
        '''
        
        l = l.reshape(-1,2)
        edge = np.zeros((l.shape[0],6,2,2),dtype=np.float32)
        
        if color:
            color_ = np.ones((l.shape[0],6),dtype=np.int)
        
        for i in range(6):
            
            edge[:, i, 0] = self.lattice_pos_array[l[:,0], l[:,1], i, :]

            
            edge[:, i, 1] = self.lattice_pos_array[l[:,0], l[:,1], (i + 1) % 6, :]

            if color:
                if i == 4:
                    color_[l[:,0] % 2 == 0, i] = -1

                if i == 1:
                    color_[l[:,0] % 2 == 1, i] = -1  
        if num:
            edge = self.lpos_to_num(edge)
        
        if color:
            return edge, color_
        else:
            return edge
#         return self.lattice_pos_array[l[:,0], l[:,1]]

        
    def lpos_to_num(self, V):
        
        # convert lattice corrdinate to index of lattice(integer)
        
        V_ = V.reshape(-1, 2)
        V_num = np.zeros(V_.shape[0], dtype=np.int)
        
        for n, v in enumerate(V_):
            V_num[n] = np.where(np.abs(self.lattice_pos - v).sum(axis=1) < self.epsilon)[0][0]
        
        
        return V_num.reshape(V.shape[:-1])
    
    
    def get_edge_color(self, edges):
        
        
        edges_ = np.sort(edges.reshape(-1,2), axis=1)
        edges_color = np.zeros(edges_.shape[0])
        
        
        for i, edge in enumerate(edges_):
            index = np.where((self.edges == edge).all(axis=1))[0][0]
            edges_color[i] = self.edges_color[index]
        
        return edges_color.reshape(edges.shape[:-1])
        
        
    
    @staticmethod
    def decompose(v, a1, a2):
        
        b = np.array([np.dot(a1,v), np.dot(a2, v)])
        
        A = np.concatenate((a1.reshape(-1,1), a2.reshape(-1,1)), axis=1)
        B = A.T @ A
        
        return np.linalg.inv(B) @ b
    
    def to_lattice_vec(self, V):
        
        V_ = V.reshape(-1,2)
        
        l_vec = np.zeros((V_.shape[0],2),dtype=np.float32)
        
        for i, v in enumerate(V_):
            w = self.decompose(v, self.a1, self.a2)
            
            l_vec[i] = w
        
        return l_vec.reshape(V.shape)
    
    
    def dimer_corr(self,l, a):
        
        '''
        l : 2 x 2 matrix l[0] = r_i is first index of hexagon, l[1] = r_j is second one
        a : 2 dimensional vector, specify which dimer which direct to a * np.pi*(1/3) will be chosen.
        
        '''
        assert l.shape[0] == 2, 'l must have exactly two edges'
        assert a.shape[0] == 2, 'a.shape[0] should be 2'
        
        assert (0<= a <=5).any()
        
        hex_edges, color = self.edges_from_hex(l = self.all_hex_index, color=True, num =True)
        

        return hex_edges[self.l[1] * l[:, 0] + l[:, 1], a ], color[self.l[1] * l[:, 0] + l[:, 1], a ]
    
    @staticmethod
    def Rotation(theta):
        return np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]]).astype(np.float32)
        

In [258]:
l =np.array([[0,0],[2,1]])
np.resize(l, (4,2))

array([[0, 0],
       [2, 1],
       [0, 0],
       [2, 1]])

In [259]:
hex_ = new_hex(l = np.array([4,2]))

In [265]:
edge, color = hex_.dimer_corr(np.array([[0,0],[2,1]]),np.array([4,2]))

In [267]:
edge

array([[8, 4],
       [1, 5]])

In [268]:
color

array([-1,  1])

In [162]:
hex_.get_edge_color(np.array([[0, 4],[4,0],[4,8]]))

array([ 1.,  1., -1.])

In [148]:
hex_.edges.shape

(24, 2)

In [149]:
hex_.edges

array([[ 0,  7],
       [ 4,  0],
       [ 8,  4],
       [ 8, 15],
       [12,  8],
       [ 0, 12],
       [ 1,  4],
       [ 5,  1],
       [ 9,  5],
       [ 9, 12],
       [13,  9],
       [ 1, 13],
       [ 2,  5],
       [ 6,  2],
       [10,  6],
       [10, 13],
       [14, 10],
       [ 2, 14],
       [ 3,  6],
       [ 7,  3],
       [11,  7],
       [11, 14],
       [15, 11],
       [ 3, 15]])